In [1]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer, AdamW, get_scheduler
import bitsandbytes as bnb
from peft import get_peft_model, LoraConfig
from datasets import load_dataset


c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.


In [ ]:
model_name = "model_name"  # Replace with the Hugging Face model name

# Load the model with 4-bit quantization
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    use_auth_token=True,  # Automatically uses the token from the environment variable
    quantization_config=bnb.QuantizationConfig(
        use_4bit=True,
        compute_dtype='float16',
        quant_type='nf4',
        nested_quant=False
    )
)

# Configure LoRA parameters
lora_config = LoraConfig(
    r=64,
    alpha=16,
    dropout=0.1
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)


In [ ]:
# Load dataset - replace "path_to_your_dataset" with your actual dataset path or identifier
dataset = load_dataset("tokenized_dataset")  # Example: "text" for a text dataset

# Tokenize dataset
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize each text in the dataset
tokenized_dataset = dataset.map(lambda x: tokenizer(x["text"]), batched=True)


In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    fp16=False,
    bf16=False,
    gradient_checkpointing=True,
    max_grad_norm=0.3,
    learning_rate=2e-4,
    weight_decay=0.001,
    lr_scheduler_type='cosine',
    warmup_ratio=0.03,
    group_by_length=True,
)


In [ ]:
# Setup optimizer
optimizer = AdamW(model.parameters(), lr=2e-4, weight_decay=0.001)

# Calculate the total number of training steps
num_training_steps = len(tokenized_dataset["train"]) // training_args.per_device_train_batch_size

# Setup scheduler
scheduler = get_scheduler(
    name='cosine',
    optimizer=optimizer,
    num_warmup_steps=int(0.03 * num_training_steps),
    num_training_steps=num_training_steps
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],  # Use the "train" split
    optimizers=(optimizer, scheduler)
)


In [ ]:
trainer.train()


In [ ]:
model.save_pretrained("./fine_tuned_llama_3")
tokenizer.save_pretrained("./fine_tuned_llama_3")
